## Import packages

In [2]:
import numpy as np
from decimal import Decimal

## Notes on preparing the data

In DAWN the data must be caked with the x-axis chosen as pixel number (pixels) and saved as a .dat file. 

It is not possible to adjust data using x-axis as the two-theta angle (degrees).

## Write out a single .esg  data file for MAUD

The current inputs assume cakes are taken from -5 to 355 degrees (right horizontal cake is zero), with cake widths of 10 degrees, giving a total of 36 cakes that start at 0 degrees. 

Note, the data is reversed during the write out, printing out intensity from the edge of the detector to the centre.

Input file:

In [3]:
file_prefix = "adc_065_TI64_NDload_900C_15mms"
dir_name = 'data/' + file_prefix + '_ascii/'
input_file = dir_name + file_prefix + '_00001.dat'

Load in the pixel number data and convert to pixel size:

In [5]:
pixel_number = np.loadtxt(input_file, usecols = (0))
print('number of pixels =', len(pixel_number))

number of pixels = 1027


In [23]:
pixel_size = 0.296

pixel_list=pixel_number*pixel_size
print('pixel spacing =', pixel_list[2] - pixel_list[1], '\u03BCm')

pixel spacing = 0.29571578080000005 μm


Load in the intensity data for each cake:

In [24]:
number_of_cakes = 36

cake_spectrum = {}
for cake in range(0, number_of_cakes):
    cake_spectrum[cake] = np.loadtxt(input_file, usecols = (cake + 1))

Write out a .esg data file using the data stored in pixel_list and cake_spectrum:

In [21]:
# user inputs
detector_distance = 1194.865
start_angle = 0
cake_width = 10

output_file = open('analysis/MAUD_065_00001.esg', 'w')

# write metadata for the top of the file
output_file.write('_pd_block_id noTitle|#0\n\
_diffrn_detector Image Plate\n\
_diffrn_detector_type Image Plate\n\
_pd_meas_step_count_time ?\n\
_diffrn_measurement_method ?\n\
_diffrn_measurement_distance_unit mm\n\
_pd_instr_dist_spec/detc {:.3f}\n'.format(detector_distance))
output_file.write('_diffrn_radiation_wavelength ?\n\
_diffrn_source_target ?\n\
_diffrn_source_power ?\n\
_diffrn_source_current ?\n\
_pd_meas_angle_omega 0.0\n\
_pd_meas_angle_chi 0.0\n\
_pd_meas_angle_phi 0.0\n\
_riet_par_spec_displac_x 0\n\
_riet_par_spec_displac_y 0\n\
_riet_par_spec_displac_z 0\n\
_riet_meas_datafile_calibrated false\n\
_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
output_file.write('loop_\n\
_pd_proc_2theta_corrected\n\
_pd_meas_intensity_total\n')

# write the first cake intensity data
for i in range(len(pixel_number)-1, -1, -1):
    output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[0][i]))

# write the other cake data with some additional info
for cake_number in range(1, number_of_cakes, 1):    
    output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
    cake_angle = start_angle + cake_number * cake_width
    output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
    output_file.write('loop_\n\
_pd_proc_2theta_corrected\n\
_pd_meas_intensity_total\n')
    
    # write the cake intensity data for each cake
    for i in range(len(pixel_number)-1, -1, -1):
        output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[cake_number][i]))
        
output_file.close()

## Loop through a larger data set and output many .esg data files for MAUD

Write out a number of .esg data files. 

The output files are written into an 'analysis' folder and saved in the format MAUD_065_00001.esg where 065 is experiment number and 00001 is test number.

In [3]:
# user inputs
detector_distance = 1194.8865
pixel_size = 0.296

# cake information
number_of_cakes = 36
start_angle = 0
cake_width = 10

# number of files
start = 0
end = 10
step = 1

# test number and input file name
test_number = 65
file_prefix = 'adc_{:03d}_TI64_NDload_900C_15mms'.format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# loop through the data using the image number of the files
for image_number in range(start, end + 1, step):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    pixel_number = np.loadtxt(input_file, usecols = (0))
    pixel_list = pixel_number * pixel_size
    cake_spectrum = {} 
    for cake in range(0, number_of_cakes):
        cake_spectrum[cake] = np.loadtxt(input_file, usecols = (cake + 1))
    
    # output MAUD data file
    output_file_name = 'analysis/MAUD_{:03d}_{:05d}.esg'.format(test_number,image_number)
    output_file = open(output_file_name, 'w')
    print(output_file_name)
    
    # write metadata for the top of the file
    output_file.write('_pd_block_id noTitle|#0\n\
_diffrn_detector Image Plate\n\
_diffrn_detector_type Image Plate\n\
_pd_meas_step_count_time ?\n\
_diffrn_measurement_method ?\n\
_diffrn_measurement_distance_unit mm\n\
_pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
    output_file.write('_diffrn_radiation_wavelength ?\n\
_diffrn_source_target ?\n\
_diffrn_source_power ?\n\
_diffrn_source_current ?\n\
_pd_meas_angle_omega 0.0\n\
_pd_meas_angle_chi 0.0\n\
_pd_meas_angle_phi 0.0\n\
_riet_par_spec_displac_x 0\n\
_riet_par_spec_displac_y 0\n\
_riet_par_spec_displac_z 0\n\
_riet_meas_datafile_calibrated false\n\
_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
    output_file.write('loop_\n\
_pd_proc_2theta_corrected\n\
_pd_meas_intensity_total\n')

    # write the first cake intensity data
    for i in range(len(pixel_number)-1,-1,-1):
        output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[0][i]))

    # write all the other cake data with additional info
    for cake_number in range(1, number_of_cakes, 1):    
        output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
        cake_angle = start_angle + cake_number * cake_width
        output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
        output_file.write('loop_\n\
_pd_proc_2theta_corrected\n\
_pd_meas_intensity_total\n')
        
        # write cake intensity data for each cake
        for i in range(len(pixel_number)-1, -1, -1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[cake_number][i]))

    output_file.close()

analysis/MAUD_065_00000.esg
analysis/MAUD_065_00001.esg
analysis/MAUD_065_00002.esg
analysis/MAUD_065_00003.esg
analysis/MAUD_065_00004.esg
analysis/MAUD_065_00005.esg
analysis/MAUD_065_00006.esg
analysis/MAUD_065_00007.esg
analysis/MAUD_065_00008.esg
analysis/MAUD_065_00009.esg
analysis/MAUD_065_00010.esg


## Output many .par analysis files for MAUD batch mode analysis

Write out a number of .par analysis files to be refined using the MAUD batch mode. 


A starting MAUD par analysis file is needed to swap the data into. This file will have been manually refined in MAUD and be setup to refine in batch mode. It is assumed that this start MAUD par file is named in the format start_065.par, where 065 is the experiment number.


The output files are written into an 'analysis' folder and saved in the format MAUD_065_00001.par where 065 is experiment number and 00001 is test number. This analysis folder will then be used in the subsequent batch mode analysis.

In [2]:
# user inputs
pixel_size = 0.296

# cake information
number_of_cakes = 36
start_angle = 0
cake_width = 10

# number of files
start = 0
end = 10
step = 1

# test number and input file name
test_number = 65
file_prefix = "adc_{:03d}_TI64_NDload_900C_15mms".format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# path to the start_***.par file
# and name of the data (.esg files) to be replaced in the MAUD start_***.par file
par_dir_name = 'data/par/'
previous_esg_filename = 'MAUD_{:03d}_00000'.format(test_number)

# leave this variable as zero
cake_number = 0

for image_number in range(start, end+1, step):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    pixel_number = np.loadtxt(input_file, usecols = (0))
    pixel_list = pixel_number * pixel_size
    cake_spectrum = {}
    for cake in range(0, number_of_cakes):
        cake_spectrum[cake] = np.loadtxt(input_file, usecols = (cake + 1))
    
    # MAUD file to swap the data into
    par_file_name = par_dir_name + 'start_{:03d}.par'.format(test_number)
    
    # output MAUD analysis file
    output_file_name = 'analysis/MAUD_{:03d}_{:05d}.par'.format(test_number, image_number)
    print(output_file_name)
    
    with open(par_file_name, 'r') as par_file, open(output_file_name, 'w') as new_par_file:
        line = par_file.readline() # note, difference between readline and readlines
        i=-1
        while line:
            i+=1
            
            # rename the datafiles in the .par file
            if previous_esg_filename in line:
                new_esg_filename = 'MAUD_{:03d}_{:05d}'.format(test_number, image_number)
                new_line = line.replace(previous_esg_filename, new_esg_filename)
                new_par_file.write(new_line)
                line = par_file.readline()
                
            # replace the data when this line is found
            if '_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma' in line and cake_number < number_of_cakes:
                new_par_file.write(line)
                line = par_file.readline()
                
                # write the new data in place (note, new data must be same length as old data)
                for j in range(len(pixel_number)-1, -1, -1):
                    new_par_file.write('{:.3f}\t{:.8f}\t1.0\n'.format(pixel_list[j], cake_spectrum[cake_number][j]))
                    line = par_file.readline()
                cake_number+=1
                i = i + len(pixel_number)
            
            # write the rest of the file
            else:
                new_par_file.write(line)
                line = par_file.readline()
                
        # reset to zero
        cake_number = 0
        
        par_file.close()    
        new_par_file.close()

analysis/MAUD_065_00000.par
analysis/MAUD_065_00001.par
analysis/MAUD_065_00002.par
analysis/MAUD_065_00003.par
analysis/MAUD_065_00004.par
analysis/MAUD_065_00005.par
analysis/MAUD_065_00006.par
analysis/MAUD_065_00007.par
analysis/MAUD_065_00008.par
analysis/MAUD_065_00009.par
analysis/MAUD_065_00010.par


## Write an instruction file for the batch mode

To run a batch mode, the instruction file should look something like this, to loop through and refine each of the analysis files (note, relative rather than absolute paths):

In [4]:
# user inputs
start = 0
end = 3
step = 1
test_number = 65

print("loop_\n\
_riet_analysis_file\n\
_riet_analysis_iteration_number\n\
_riet_analysis_wizard_index\n\
_riet_analysis_fileToSave\n\
_riet_append_simple_result_to\n\
_riet_append_result_to\n\n")
# note, _riet_meas_datafile_name\n\ removed from above for swapping the files.

for i in range (start, end + 1, step):

# note, one slash \ stops formatting i.e. \\t prints \t
# for curly brackets add two, i.e. print({{) prints out {

    print("""'/analysis/MAUD_{:03d}_{:05d}.par' 7 13 '/analysis/MAUD_{:03d}_{:05d}_refined.par' '/analysis/batch_results_simple.txt' '/analysis/batch_results.txt\'\n""".format(test_number, i, test_number, i, test_number, i))

loop_
_riet_analysis_file
_riet_analysis_iteration_number
_riet_analysis_wizard_index
_riet_analysis_fileToSave
_riet_append_simple_result_to
_riet_append_result_to


'/analysis/MAUD_065_00000.par' 7 13 '/analysis/MAUD_065_00000_refined.par' '/analysis/batch_results_simple.txt' '/analysis/batch_results.txt'

'/analysis/MAUD_065_00001.par' 7 13 '/analysis/MAUD_065_00001_refined.par' '/analysis/batch_results_simple.txt' '/analysis/batch_results.txt'

'/analysis/MAUD_065_00002.par' 7 13 '/analysis/MAUD_065_00002_refined.par' '/analysis/batch_results_simple.txt' '/analysis/batch_results.txt'

'/analysis/MAUD_065_00003.par' 7 13 '/analysis/MAUD_065_00003_refined.par' '/analysis/batch_results_simple.txt' '/analysis/batch_results.txt'



Write out an instruction file (.ins) defining the batch mode loop. Stored in a file called 'batch files'

In [3]:
# user inputs
start = 0
end = 10
step = 1
test_number = 65

batch_file = open('batch files/MAUD_batch_065.ins','w')

batch_file.write("loop_\n\
_riet_analysis_file\n\
_riet_analysis_iteration_number\n\
_riet_analysis_wizard_index\n\
_riet_analysis_fileToSave\n\
_riet_append_simple_result_to\n\n")
# note, _riet_meas_datafile_name\n\ removed from above for swapping the files.
# note, also _riet_append_result_to\n\ but this is only the Rwp value

for i in range (start, end + 1, step):

# note, one slash \ stops formatting i.e. \\t prints \t
# for curly brackets add two, i.e. print({{) prints out {
    
    batch_file.write("""'/analysis/MAUD_{:03d}_{:05d}.par' 7 13 '/analysis/MAUD_{:03d}_{:05d}_refined.par' '/analysis/batch_results.txt\'\n""".format(test_number, i, test_number, i, test_number, i))

batch_file.close()

## Running MAUD batch mode

Place the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in a sensible location, such as; 


**/Users/mbcx9cd4/Documents/maud/batch/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


In the terminal navigate to the hidden files within the maud.app using the command; 


**cd /Applications/Maud.app/Contents/Resources/Java**


_Note, you can view these files by right clicking on show package contents._


Then, run the command to start the batch mode;


**java -mx2048M -cp Maud.jar:miscLib.jar:jgaec.jar:ij.jar com.radiographema.MaudText -f /Users/mbcx9cd4/Documents/maud/batch/MAUD_batch_065_mac.ins**


A pop-up window will then appear, navigate to the .ins instruction file and click open. The batch mode will then run, showing the refinements in the terminal.

## Plotting refinement results

In [ ]:
results_file = 'analysis/batch_results.txt'
results = np.loadtxt(result_file, usecols = (0), skiprows=1)
print('number of pixels =', len(pixel_number))

## Saving ODF from MAUD analysis files for plotting in MTEX